# Implicit Trajectory combined with RRT #
- Work space: 3D with obstacles
- Obstacle type: static and known
- Agent Model: Dubins airplane (Owen2015)
- Integration Algorithm: 4th Order Runge Kutta
- Implicit Curve: Time-invariant Circle
- Distance Metric: admissionable heuristic, based on time-optimal dubins-airplane paths (altitude difference is integrated in a linear fashion)

In [21]:
# load the code
%run 'MAIN_CODE.ipynb'
import random

# Sensitivity Experiments

## VF-RRT+impl.Curve: $E_{s}$ vs $N_\lambda$

### Params

In [22]:
import time

#Parameters in General
qstart = np.array([5.,5.,1.,np.pi])# specific initial point
MaxNodes = 150
MAXiter = MaxNodes*3

# Experiment parameters for VF-RRT
lamSteps = 11
lambdaNsteps_range=np.linspace(5,15,lamSteps)
EsSteps=10
Es_range=np.linspace(0.05,0.95,EsSteps)
lamdb=1.0 #w/e

### Obstacles

In [23]:
#specific obstacles, fixed, known, not moving

# reset the RNG for deterministic obstacles
random.seed( 199 ) #arbitrary

Obstacles = CreateSphericalObstacles(N=70,r=4.)
while isBlocked(qstart, Obstacles):
    Obstacles = CreateSphericalObstacles(N=70,r=4.)

### Experiment

In [24]:
# initilize
VFRRTupstream = np.zeros((EsSteps, lamSteps))
VFRRTnodes = np.zeros((EsSteps, lamSteps))
VFRRTexetime = np.zeros((EsSteps, lamSteps))
VFRRTwork = np.zeros((EsSteps, lamSteps))
VFRRTsuccess = np.zeros((EsSteps, lamSteps))
VFRRTiter = np.zeros((EsSteps, lamSteps))
VFRRTeuclLength = np.zeros((EsSteps, lamSteps))
VFRRTanglLength = np.zeros((EsSteps, lamSteps))

print('VFRRT')
print('Es\tlamStps\tt\tsuccess\twork\tupstr\tnodes\titer\teuDist\tangDist\thiLim\tloLim')
for i in range(0,EsSteps):
    Es = Es_range[i]
    for j in range(0,lamSteps):
        lambdaNsteps = lambdaNsteps_range[j]
        
        # reset the RNG
        random.seed( 500 ) #arbitrary
        
        # Execute and return Tree
       
        stampBegin = time.process_time()
        T = VFRRT(qstart,VF,lambdaNsteps,Es,lamdb,MaxNodes,MAXiter,Obstacles,bounds=bounds)
        #while not T.successful:
        #    T = VFRRT(qstart,VF,lambdaNsteps,Es,lamdb,MaxNodes,MAXiter,Obstacles,bounds=bounds)
        CPUtimePassed = time.process_time()-stampBegin
        
        while not T.successful:
            stampBegin = time.process_time()
            T = VFRRT(qstart,VF,lambdaNsteps,Es,lamdb,MaxNodes,MAXiter,Obstacles,bounds=bounds)
            CPUtimePassed = time.process_time()-stampBegin
        
       
        if T.successful:
            VFRRTexetime[i][j] = CPUtimePassed
            VFRRTiter[i][j] = T.iterations
            VFRRTsuccess[i][j] = 1
            IDpath,Datapath,Edgepath = T.get_solution()
            work,upstream = T.get_WorkAndUpstreamOfPath(Datapath)
            euclDist,anglDist = T.get_length(Datapath)
            VFRRTnodes[i][j] = T.NumNodes
            VFRRTupstream[i][j] = upstream
            VFRRTwork[i][j] = work
            VFRRTeuclLength[i][j] = euclDist
            VFRRTanglLength[i][j] = anglDist

            print('%.3f\t%d\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\t%s\t%s' % (Es,lambdaNsteps,CPUtimePassed,work,upstream,T.NumNodes,T.iterations,euclDist,anglDist,str(T.lambd_high),str(T.lambd_low)))
        else:
            print('%.3f\t%d\t%.3f\tFalse\t\t\t%d\t%d\t\t\t%s\t%s' % (Es,lambdaNsteps,CPUtimePassed,T.NumNodes,T.iterations,str(T.lambd_high),str(T.lambd_low)))

        #T.plot(DrawTrajectory=True,DrawCollisionNodes=True)
        #T.plot(obs=Obstacles,OnlyClosestObs=True,DrawCollisionNodes=True)
        #DrawStats(T)



VFRRT
Es	lamStps	t	success	work	upstr	nodes	iter	euDist	angDist	hiLim	loLim
0.050	5	0.626	True	4.293	2.887	114	122	4.599	17.621	False	True
0.050	6	0.589	True	4.293	2.894	114	122	4.599	17.531	False	True
0.050	7	0.584	True	4.299	2.830	114	122	4.599	17.344	False	True
0.050	8	0.553	True	4.295	2.870	114	122	4.599	17.306	False	False
0.050	9	0.547	True	4.317	2.654	113	122	4.598	16.792	False	False
0.050	10	0.686	True	4.314	2.677	113	122	4.598	16.741	False	False
0.050	11	0.608	True	4.320	2.618	113	122	4.598	16.624	False	False
0.050	12	0.613	True	4.323	2.582	113	122	4.597	16.533	False	False
0.050	13	0.469	True	4.342	2.100	101	101	4.567	12.862	False	False
0.050	14	0.592	True	4.291	2.457	135	135	4.542	11.784	False	False
0.050	15	0.344	True	4.379	1.581	94	94	4.544	13.458	False	False
0.150	5	0.531	True	4.295	2.868	114	122	4.599	17.473	False	True
0.150	6	0.506	True	4.309	2.727	113	122	4.598	17.023	False	False
0.150	7	0.560	True	4.367	4.324	124	132	4.835	14.856	False	False
0.150	8	0.440	True	4.368	1.8

### Plots


In [25]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTeuclLength

z = ma.masked_where(z <= 0, z)


fig = plt.figure()
cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
plt.xlabel('lambda steps')
plt.ylabel('Es')
plt.title('VFRRTeuclLength')
cbar = plt.colorbar()





In [26]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTnodes

#z = ma.masked_where(z >= MaxNodes, z)
z = ma.masked_where(z <= 0, z)


fig = plt.figure()
cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
plt.xlabel('lambda steps')
plt.ylabel('Es')

plt.title('VFRRTnodes')
cbar = plt.colorbar()





In [27]:


import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTsuccess

#z = ma.masked_where(z >= MaxNodes, z)
z = ma.masked_where(z <= 0, z)


#fig = plt.figure()
#cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
#plt.xlabel('lambda steps')
#plt.ylabel('Ein')

#plt.title('VFRRTsuccess')
#cbar = plt.colorbar()





In [28]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTwork

#z = ma.masked_where(z >= MaxNodes, z)
z = ma.masked_where(z <= 0, z)


fig = plt.figure()
cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
plt.xlabel('lambda steps')
plt.ylabel('Es')

plt.title('VFRRTwork')
cbar = plt.colorbar()





In [29]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTupstream

#z = ma.masked_where(z >= MaxNodes, z)
z = ma.masked_where(z <= 0, z)


fig = plt.figure()
cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
plt.xlabel('lambda steps')
plt.ylabel('Es')

plt.title('VFRRTupstream')
cbar = plt.colorbar()



#plt.show()

In [30]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

x = lambdaNsteps_range
y = Es_range

X, Y = np.meshgrid(x, y)

# A low hump with a spike coming out of the top right.
# Needs to have z/colour axis on a log scale so we see both hump and spike.
# linear scale only shows the spike.
z = VFRRTexetime

#z = ma.masked_where(z >= MaxNodes, z)
z = ma.masked_where(z <= 0, z)


fig = plt.figure()
cs = plt.contourf(X, Y, z, cmap=cm.PuBu_r)
plt.xlabel('lambda steps')
plt.ylabel('Es')

plt.title('VFRRTexetime')
cbar = plt.colorbar()



plt.show()